In [784]:
import requests
from datetime import date
import datetime
import PIL
from PIL import Image
import PIL.ImageOps
import os
import numpy
from numpy import asarray
import pandas as pd
from requests.exceptions import ConnectionError
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import os.path
from os import path
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import LeakyReLU

In [496]:
# Save Image

def save_image(image, path):
    if os.path.exists(path):
        os.remove(path)
    
    image.save(path, 'JPEG')

def getdate(i):
    p_date = date.today()
    days = datetime.timedelta(i+1)
    e_date = p_date - days
    n_date = e_date.strftime("%Y-%m-%d")
    return n_date

def openImage(i):
    extension = '.JPEG'
    n_date = getdate(i+10)
    file_name = n_date + extension
    image = PIL.Image.open(file_name)
    return image

In [3]:
def download_images(t_images):
    try:
        for i in range(t_images):
            n_date = getdate(i)
            f_name = n_date+'.JPEG'
            if(path.isfile(f_name)):
                continue
            else:
                URL_1 = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor&CRS=EPSG:4326&TIME='
                URL_2 = '&WRAP=DAY&BBOX=5.505452,79.264304,10.049224,82.491415&FORMAT=image/jpeg&WIDTH=367&HEIGHT=517&AUTOSCALE=TRUE&ts=1604254628320'
                URL = URL_1+n_date+URL_2
                r = requests.get(URL,allow_redirects=True)
                open(f_name,'wb').write(r.content)
                print('saved ' + f_name)
    except ConnectionError:
        print("Check your Internet Connection and try again")

In [464]:
def convert_2_BnW(t_images):
    train_images = []
    filelist = list()
    for i in range(t_images):
        n_date = getdate(i)
        image = openImage(i)
        greyscale_img = image.convert(mode="1", dither=Image.NONE)
        greyscale_img = greyscale_img.convert('L')
        inverted_image = PIL.ImageOps.invert(greyscale_img)
        n_name = n_date+'.JPEG'
        save_image(inverted_image,n_name)
#         print(n_name,end =' ')
        image_array = asarray(image)
#         print(i,end=' ')
#         print(image_array.shape)
        if((image_array.shape)==(517,364)):
            image_array = image_array.reshape(1,517,364)
            filelist.insert(i,n_name)
#             print(i)
            if(i==0):
                train_images = image_array
    #             print(train_images.shape)
            else:
                train_images = numpy.vstack((train_images,image_array))
#                 print(train_images.shape)
        else:
            continue
    train_images = train_images.reshape(-1, 188188)
    return train_images,filelist

In [504]:
def count_blk_cover(i):
#     n_date = getdate(i+1)
    im = PIL.Image.open(i)
    count_blacks = 0
    count_whites = 0
    width, height = im.size
    t_px=width*height

    for h in range(height):
        string = ''
        for w in range(width):
            cordinates = w,h
            val = im.getpixel(cordinates)
            if(val>50):
                count_blacks += 1
    return count_blacks

def categorize_clouds(count_blacks):
    t_px = 188188
    cloud_category = numpy.array([])
    if (count_blacks/t_px)<0.25:
        cloud_category = '1'
    elif (count_blacks/t_px)<0.5:
        cloud_category = '2'
    elif (count_blacks/t_px)<0.75:
        cloud_category = '3'
    else:
        cloud_category = '4'
    return cloud_category


In [505]:
def categorize(file_list):
    cloud_cats = numpy.array([],dtype='U')
    for j in file_list:
        black_count = count_blk_cover(j)
        category = categorize_clouds(black_count)
        cloud_cats = numpy.insert(cloud_cats,len(cloud_cats),category)
        print(j)
    return cloud_cats

In [499]:
download_images(1811)
train_images = []
filelist = []
cloud_cats = []
train_images,filelist = convert_2_BnW(1500)

In [ ]:
test_images = []
t_fl = []
test_images,t_fl = convert_2_BnW(200)
test_cloud_cats = categorize(t_fl)

In [500]:
train_images.shape

(1496, 188188)

In [506]:
cloud_cats = categorize(filelist)

2020-11-03.JPEG
2020-11-02.JPEG
2020-11-01.JPEG
2020-10-31.JPEG
2020-10-30.JPEG
2020-10-29.JPEG
2020-10-28.JPEG
2020-10-27.JPEG
2020-10-26.JPEG
2020-10-25.JPEG
2020-10-24.JPEG
2020-10-23.JPEG
2020-10-22.JPEG
2020-10-21.JPEG
2020-10-20.JPEG
2020-10-19.JPEG
2020-10-18.JPEG
2020-10-17.JPEG
2020-10-16.JPEG
2020-10-15.JPEG
2020-10-14.JPEG
2020-10-13.JPEG
2020-10-12.JPEG
2020-10-11.JPEG
2020-10-10.JPEG
2020-10-09.JPEG
2020-10-08.JPEG
2020-10-07.JPEG
2020-10-06.JPEG
2020-10-05.JPEG
2020-10-04.JPEG
2020-10-03.JPEG
2020-10-02.JPEG
2020-10-01.JPEG
2020-09-30.JPEG
2020-09-29.JPEG
2020-09-28.JPEG
2020-09-27.JPEG
2020-09-26.JPEG
2020-09-25.JPEG
2020-09-24.JPEG
2020-09-23.JPEG
2020-09-22.JPEG
2020-09-21.JPEG
2020-09-20.JPEG
2020-09-19.JPEG
2020-09-18.JPEG
2020-09-17.JPEG
2020-09-16.JPEG
2020-09-15.JPEG
2020-09-14.JPEG
2020-09-13.JPEG
2020-09-12.JPEG
2020-09-11.JPEG
2020-09-10.JPEG
2020-09-09.JPEG
2020-09-08.JPEG
2020-09-07.JPEG
2020-09-06.JPEG
2020-09-05.JPEG
2020-09-04.JPEG
2020-09-03.JPEG
2020-09-

2019-06-09.JPEG
2019-06-08.JPEG
2019-06-07.JPEG
2019-06-06.JPEG
2019-06-05.JPEG
2019-06-04.JPEG
2019-06-03.JPEG
2019-06-02.JPEG
2019-06-01.JPEG
2019-05-31.JPEG
2019-05-30.JPEG
2019-05-29.JPEG
2019-05-28.JPEG
2019-05-27.JPEG
2019-05-26.JPEG
2019-05-25.JPEG
2019-05-24.JPEG
2019-05-23.JPEG
2019-05-22.JPEG
2019-05-21.JPEG
2019-05-20.JPEG
2019-05-19.JPEG
2019-05-18.JPEG
2019-05-17.JPEG
2019-05-16.JPEG
2019-05-15.JPEG
2019-05-14.JPEG
2019-05-13.JPEG
2019-05-12.JPEG
2019-05-11.JPEG
2019-05-10.JPEG
2019-05-09.JPEG
2019-05-08.JPEG
2019-05-07.JPEG
2019-05-06.JPEG
2019-05-05.JPEG
2019-05-04.JPEG
2019-05-03.JPEG
2019-05-02.JPEG
2019-05-01.JPEG
2019-04-30.JPEG
2019-04-29.JPEG
2019-04-28.JPEG
2019-04-27.JPEG
2019-04-26.JPEG
2019-04-25.JPEG
2019-04-24.JPEG
2019-04-23.JPEG
2019-04-22.JPEG
2019-04-21.JPEG
2019-04-20.JPEG
2019-04-19.JPEG
2019-04-18.JPEG
2019-04-17.JPEG
2019-04-16.JPEG
2019-04-15.JPEG
2019-04-14.JPEG
2019-04-13.JPEG
2019-04-12.JPEG
2019-04-11.JPEG
2019-04-10.JPEG
2019-04-09.JPEG
2019-04-

2018-01-12.JPEG
2018-01-11.JPEG
2018-01-10.JPEG
2018-01-09.JPEG
2018-01-08.JPEG
2018-01-07.JPEG
2018-01-06.JPEG
2018-01-05.JPEG
2018-01-04.JPEG
2018-01-03.JPEG
2018-01-02.JPEG
2018-01-01.JPEG
2017-12-31.JPEG
2017-12-30.JPEG
2017-12-29.JPEG
2017-12-28.JPEG
2017-12-27.JPEG
2017-12-26.JPEG
2017-12-25.JPEG
2017-12-24.JPEG
2017-12-23.JPEG
2017-12-22.JPEG
2017-12-21.JPEG
2017-12-20.JPEG
2017-12-19.JPEG
2017-12-18.JPEG
2017-12-17.JPEG
2017-12-16.JPEG
2017-12-15.JPEG
2017-12-14.JPEG
2017-12-13.JPEG
2017-12-12.JPEG
2017-12-11.JPEG
2017-12-10.JPEG
2017-12-09.JPEG
2017-12-08.JPEG
2017-12-07.JPEG
2017-12-06.JPEG
2017-12-05.JPEG
2017-12-04.JPEG
2017-12-03.JPEG
2017-12-02.JPEG
2017-12-01.JPEG
2017-11-30.JPEG
2017-11-29.JPEG
2017-11-28.JPEG
2017-11-27.JPEG
2017-11-26.JPEG
2017-11-25.JPEG
2017-11-24.JPEG
2017-11-23.JPEG
2017-11-22.JPEG
2017-11-21.JPEG
2017-11-20.JPEG
2017-11-19.JPEG
2017-11-18.JPEG
2017-11-17.JPEG
2017-11-16.JPEG
2017-11-15.JPEG
2017-11-14.JPEG
2017-11-13.JPEG
2017-11-12.JPEG
2017-11-

In [509]:
(unique, counts) = numpy.unique(cloud_cats, return_counts=True)
frequencies = numpy.asarray((unique, counts)).T
t_classes = int(len(unique))+1

In [510]:
print(frequencies)

[['1' '331']
 ['2' '393']
 ['3' '416']
 ['4' '356']]


In [932]:
# def run_model():
#building the model
# 3 layers, 2 layers with 64 neurons and relu function
# 1 layer with 10 neurons and softmax function
model = Sequential()
model.add(Dense(128, activation = 'sigmoid',input_dim=188188))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(t_classes, activation = 'softmax'))

#Compile the model
#The loss function measures how well the model did on training, and the tries to improve on it using the optimizer
model.compile(
    optimizer='RMSprop',
    loss = 'categorical_crossentropy', #Classes that are greater than 2
    metrics = ['accuracy']
)

In [933]:
#fitting the model
model.fit(
    train_images,
        to_categorical(cloud_cats), #Ex. 2 it expects [0,0,1,0,0,0,0,0,0]
        epochs = 20, #number of iterations over the entire dataset to train on
        batch_size = 64, #number of samples per gradient update for training   
        validation_data=(test_images,to_categorical(test_cloud_cats))
)

Train on 1496 samples, validate on 200 samples
Epoch 1/20
1496/1496 [==============================] - 16s 11ms/step - loss: 1.3650 - acc: 0.3476 - val_loss: 1.3311 - val_acc: 0.2850
Epoch 2/20
1496/1496 [==============================] - 12s 8ms/step - loss: 1.1967 - acc: 0.4686 - val_loss: 1.1541 - val_acc: 0.5150
Epoch 3/20
1496/1496 [==============================] - 12s 8ms/step - loss: 1.1123 - acc: 0.5107 - val_loss: 1.0516 - val_acc: 0.5850
Epoch 4/20
1496/1496 [==============================] - 13s 9ms/step - loss: 1.0155 - acc: 0.5575 - val_loss: 0.8603 - val_acc: 0.7100
Epoch 5/20
1496/1496 [==============================] - 13s 9ms/step - loss: 0.9508 - acc: 0.5729 - val_loss: 1.0265 - val_acc: 0.3900
Epoch 6/20
1496/1496 [==============================] - 13s 9ms/step - loss: 0.9153 - acc: 0.5842 - val_loss: 0.7954 - val_acc: 0.6450
Epoch 7/20
1496/1496 [==============================] - 13s 9ms/step - loss: 0.9091 - acc: 0.5749 - val_loss: 0.8300 - val_acc: 0.5250
Epoch 8

In [934]:
yhat = model.predict_classes(test_images)
# print(yhat)

In [935]:
# print(test_cloud_cats)

In [936]:
scores = model.evaluate(test_images,to_categorical(test_cloud_cats), verbose=0)

In [937]:
print("Total Loss : "+str(scores[0]))
print("Total Accuracy : "+str(scores[1]))

Total Loss : 0.586857647895813
Total Accuracy : 0.805


In [938]:
(unique, counts) = numpy.unique(yhat, return_counts=True)
frequencies_yhat = numpy.asarray((unique, counts)).T
print(frequencies_yhat)

[[ 1 70]
 [ 2 57]
 [ 3 20]
 [ 4 53]]


In [939]:
(unique, counts) = numpy.unique(test_cloud_cats, return_counts=True)
frequencies_test = numpy.asarray((unique, counts)).T
print(frequencies_test)

[[ 1 58]
 [ 2 43]
 [ 3 45]
 [ 4 54]]


In [940]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix((test_cloud_cats.astype(int)),yhat)
print(cm)

[[57  1  0  0]
 [12 31  0  0]
 [ 1 24 20  0]
 [ 0  1  0 53]]


In [941]:
print(classification_report(test_cloud_cats, yhat,labels=unique))

              precision    recall  f1-score   support

           1       0.81      0.98      0.89        58
           2       0.54      0.72      0.62        43
           3       1.00      0.44      0.62        45
           4       1.00      0.98      0.99        54

    accuracy                           0.81       200
   macro avg       0.84      0.78      0.78       200
weighted avg       0.85      0.81      0.80       200



In [931]:
#RMSprop Total Accuracy : 0.6 #predictions are good
#Adadelta Total Accuracy : 0.43 #didn't predict category 3 at all #Don't recommend for this model
#Adagrad Total Accuracy : 0.65 #Really good predictions
#Adamax Total Accuracy : 0.65 #Really good predictions cat 4
#Nadam Toatal Accuracy : 0.675 #Really good prediction except for cat 2